In [5]:
import os
print(os.getcwd())

/Users/viktorsuter/Desktop/discogs/notebooks


In [2]:
import os
from dotenv import load_dotenv

# Load from the project root
load_dotenv('../.env')

DISCOGS_TOKEN = os.getenv('DISCOGS_TOKEN')
print("Token loaded!" if DISCOGS_TOKEN else "Failed!")

Failed!
